In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

[]


In [2]:
import bq_helper
from bq_helper import BigQueryHelper

chicago_taxi = bq_helper.BigQueryHelper(active_project="bigquery-public-data", dataset_name="chicago_taxi_trips")
bq_assistant = BigQueryHelper("bigquery-public-data", "chicago_taxi_trips")
bq_assistant.list_tables()
bq_assistant.head("taxi_trips", num_rows=3)
bq_assistant.table_schema("taxi_trips")

Using Kaggle's public dataset BigQuery integration.
Using Kaggle's public dataset BigQuery integration.


,name,type,mode,description
0,unique_key,STRING,REQUIRED,Unique identifier for the trip.
1,taxi_id,STRING,REQUIRED,A unique identifier for the taxi.
2,trip_start_timestamp,TIMESTAMP,NULLABLE,"When the trip started, rounded to the nearest ..."
3,trip_end_timestamp,TIMESTAMP,NULLABLE,"When the trip ended, rounded to the nearest 15..."
4,trip_seconds,INTEGER,NULLABLE,Time of the trip in seconds.
5,trip_miles,FLOAT,NULLABLE,Distance of the trip in miles.
6,pickup_census_tract,INTEGER,NULLABLE,The Census Tract where the trip began. For pri...
7,dropoff_census_tract,INTEGER,NULLABLE,The Census Tract where the trip ended. For pri...
8,pickup_community_area,INTEGER,NULLABLE,The Community Area where the trip began.
9,dropoff_community_area,INTEGER,NULLABLE,The Community Area where the trip ended.


In [3]:
query1 = '''
SELECT 
  EXTRACT(DAYOFWEEK FROM trip_start_timestamp) AS day,
  FORMAT('%3.2f', MAX(fare)) AS maximum_fare,
  FORMAT('%3.2f', MIN(fare)) AS minimum_fare,
  FORMAT('%3.2f', AVG(fare)) AS avg_fare,
  FORMAT('%3.2f', STDDEV(fare)) AS std_dev_fare,
  COUNT(1) AS rides
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE trip_seconds >= 240 AND fare > 0 AND fare <600
GROUP BY
  day
ORDER BY
  day
'''
response1 = chicago_taxi.query_to_pandas_safe(query1, max_gb_scanned=10)
response1.head(10)

,day,maximum_fare,minimum_fare,avg_fare,std_dev_fare,rides
0,1,599.50,0.01,14.17,12.17,18777330
1,2,599.00,0.01,14.48,13.01,19551750
2,3,599.85,0.01,13.58,12.28,21276337
3,4,599.65,0.01,13.69,12.34,22402276
4,5,597.00,0.01,13.71,12.29,23883600
5,6,599.85,0.01,13.10,11.48,25936498
6,7,592.50,0.01,12.06,9.85,23285519


In [4]:
query2 = """SELECT
  dropoff_community_area,
  FORMAT('%3.2f', AVG(tips)) AS average_tip,
  FORMAT('%3.2f', MAX(tips)) AS max_tip
FROM
  `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
  dropoff_community_area IS NOT NULL
GROUP BY
  dropoff_community_area
ORDER BY
  average_tip DESC
LIMIT
  10
        """
response2 = chicago_taxi.query_to_pandas_safe(query2, max_gb_scanned=10)
response2.head(10)

,dropoff_community_area,average_tip,max_tip
0,76,3.99,596.85
1,72,3.45,80.00
2,56,3.37,999.99
3,74,2.40,150.00
4,41,2.07,250.00
5,9,1.99,74.75
6,64,1.86,100.00
7,75,1.67,75.00
8,39,1.57,212.50
9,12,1.56,220.00


In [5]:
query3 = """SELECT
  FORMAT('%02.0fm to %02.0fm', min_minutes, max_minutes) AS minutes_range,
  SUM(trips) AS total_trips,
  FORMAT('%3.2f', SUM(total_fare) / SUM(trips)) AS average_fare
FROM (
  SELECT
    MIN(duration_in_minutes) OVER (quantiles) AS min_minutes,
    MAX(duration_in_minutes) OVER (quantiles) AS max_minutes,
    SUM(trips) AS trips,
    SUM(total_fare) AS total_fare
  FROM (
    SELECT
      ROUND(trip_seconds / 60) AS duration_in_minutes,
      NTILE(10) OVER (ORDER BY trip_seconds / 60) AS quantile,
      COUNT(1) AS trips,
      SUM(fare) AS total_fare
    FROM
      `bigquery-public-data.chicago_taxi_trips.taxi_trips`
    WHERE
      ROUND(trip_seconds / 60) BETWEEN 1 AND 90
    GROUP BY
      trip_seconds,
      duration_in_minutes )
  GROUP BY
    duration_in_minutes,
    quantile
  WINDOW quantiles AS (PARTITION BY quantile)
  )
GROUP BY
  minutes_range
ORDER BY
  Minutes_range
        """
response3 = chicago_taxi.query_to_pandas_safe(query3, max_gb_scanned=10)
response3.head(10)

,minutes_range,total_trips,average_fare
0,01m to 09m,82349273,6.60
1,10m to 18m,51844471,11.46
2,19m to 27m,17093934,23.26
3,28m to 36m,8220184,32.66
4,37m to 45m,4481673,38.20
5,46m to 54m,2514577,41.57
6,55m to 63m,1345795,44.26
7,64m to 72m,667515,46.78
8,73m to 81m,318023,49.93
9,82m to 90m,150048,52.98


In [6]:

modelquery = """CREATE OR REPLACE MODEL chicagotaxi.taxifare_model_1
OPTIONS
  (model_type='linear_reg', labels=['trip_total']) AS


WITH params AS (
    SELECT
    1 AS TRAIN,
    2 AS EVAL
    ),
  taxitrips AS (
  SELECT
    trip_seconds,
    trip_miles,
    trip_total,
    EXTRACT(HOUR FROM trip_start_timestamp) AS hourofday,
    SQRT(POW((pickup_longitude - dropoff_longitude),2) + POW(( pickup_latitude - dropoff_latitude), 2)) as dist, 
    SQRT(POW((pickup_longitude - dropoff_longitude),2)) as longitude, 
    SQRT(POW((pickup_latitude - dropoff_latitude), 2)) as latitude 
  FROM
    `bigquery-public-data.chicago_taxi_trips.taxi_trips`, params
WHERE trip_seconds > 0 AND trip_total BETWEEN 3 and 200
    AND MOD(ABS(FARM_FINGERPRINT(CAST(trip_start_timestamp AS STRING))),1000) = params.TRAIN
  )

  SELECT *
  FROM taxitrips
        """

In [7]:
evalquery = """SELECT
  SQRT(mean_squared_error) AS rmse
FROM
  ML.EVALUATE(MODEL chicagotaxi.taxifare_model_1,
  (

WITH params AS (
    SELECT
    1 AS TRAIN,
    2 AS EVAL
    ),
  taxitrips AS (
  SELECT
    trip_seconds,
    trip_miles,
    trip_total,
    EXTRACT(HOUR FROM trip_start_timestamp) AS hourofday,
    SQRT(POW((pickup_longitude - dropoff_longitude),2) + POW(( pickup_latitude - dropoff_latitude), 2)) as dist, 
    SQRT(POW((pickup_longitude - dropoff_longitude),2)) as longitude, 
    SQRT(POW((pickup_latitude - dropoff_latitude), 2)) as latitude 
  FROM
    `bigquery-public-data.chicago_taxi_trips.taxi_trips`, params
WHERE trip_seconds > 0 AND trip_total BETWEEN 3 and 200
    AND MOD(ABS(FARM_FINGERPRINT(CAST(trip_start_timestamp AS STRING))),1000) = params.EVAL
  )

  SELECT *
  FROM taxitrips

  ))
        """